In [31]:
import cv2
import numpy as np
import pandas as pd
from pathlib import Path
import PySimpleGUI as sg
import yaml

In [ ]:
draw_text(text,
    location,
    color = "black",
    font = None,
    angle = 0,
    text_location = "center")

In [13]:
w = 200 * 3
h = 20
d_w = 20.0
s = 0

c = None
txt = None

layout = [
    [sg.Graph((w, h), (0,0), (w, h),  background_color='lime', key='-P-')],
    [sg.Button('+', key='-A-')],
    [sg.Button('S', key='-S-'), [sg.Button('E', key='-E-')],],
]

window = sg.Window('App', layout, finalize=True)


def draw_rect(s):
    c1 = s * d_w
    s += 1
    c2 = s * d_w
    
    window['-P-'].draw_rectangle(
        (c1, 0),
        (c2, h),
        fill_color = c
    )
    return s

  
def draw_txt(s):
    x = (s - 1) * d_w + d_w / 2
    
    window['-P-'].draw_text(
        txt,
        (x, h / 2),
    )


while True:
    event, values = window.read()
    
    if event == sg.WIN_CLOSED:
        break
    
    if event == '-A-':
        s = draw_rect(s)
        draw_txt(s)
        
    if event == '-S-':
        c = 'orange'
        txt = 'S'
        
    if event == '-E-':
        c = 'red'
        txt = 'E'
        
    window.read(timeout=0)
        
window.close()

In [12]:
%pip install pyyaml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.8/661.8 kB 5.4 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


# kéne egy olyan funkció, amivel sub actionöket lehet csinálni

In [93]:
def load_config():
    with open(Path.cwd().joinpath('config.yaml'), 'r') as f:
        y = yaml.load(f, yaml.FullLoader)
    return y

def get_pos(cap):
    return int(cap.get(cv2.CAP_PROP_POS_FRAMES))


def update_duration(window, cap):
    window['-DURATION-'].update(
        f'{round(cap.get(cv2.CAP_PROP_FRAME_COUNT) / cap.get(cv2.CAP_PROP_FPS), 3)} s'
    )


def encode_as_bytes(f, w, h):
    f = cv2.resize(f, (w, h))
    return cv2.imencode('.ppm', f)[1].tobytes()


def is_label_valid(label):
    for inf in label[:-1]:
        if not inf:
            return False
    return True


def update_seconds(window, cap):
    pos = cap.get(cv2.CAP_PROP_POS_FRAMES)
    fps = cap.get(cv2.CAP_PROP_FPS)
    seconds = round(pos / fps, 3)
    window['-SECONDS-'].update(f'{seconds} s')

   
def update_image(window, cap, w, h):
    ret, f = cap.read()
    if not ret:
        sg.popup('No more frames left!')
        return
    fbytes = encode_as_bytes(f, w, h)
    window['-IMAGE-'].update(data=fbytes)
    
    
def update_play_button(window, is_playing):
    window['-PLAY-'].update('Pause' if is_playing else 'Play')


def load_video(p, conf):
    cap = cv2.VideoCapture(str(p))
    return cap
    
    
def load_from_beginning(window, cap, w, h):
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    ret, f = cap.read()
    fbytes = encode_as_bytes(f, w, h)
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    window['-IMAGE-'].update(data=fbytes)
    

def load_prev(window, cap, w, h):
    t = cap.get(cv2.CAP_PROP_POS_FRAMES)
    cap.set(cv2.CAP_PROP_POS_FRAMES, t - 2)
    ret, f = cap.read()
    fbytes = encode_as_bytes(f, w, h)
    window['-IMAGE-'].update(data=fbytes)

   
def load_next(window, cap, w, h):
    ret, f = cap.read()
    fbytes = encode_as_bytes(f, w, h)
    window['-IMAGE-'].update(data=fbytes)

    
def draw_timestep_on_progressbar(window, cap, unit, h, txt, color):
    pos = get_pos(cap)
            
    x1 = (pos - 1) * unit
    y1 = h
    x2 = pos * unit
    y2 = 0
    
    window['-PROGRESS-'].draw_rectangle(
        (x1, y1),
        (x2, y2),
        fill_color=color,
        line_color = color,
        line_width=2,
    )
    
    x = x1 + unit / 2
    y = h + h / 2
    
    window['-PROGRESS-'].draw_text(
        txt,
        (x, y),
    )
    
    
def update_table(window, tvalues):
    window['-TABLE-'].update(tvalues)
    
    
def update_counter(window, cap):
    window['-COUNTER-'].update(get_pos(cap))
    

def app():
    img_w = 500
    img_h = 500
    
    b_width = img_w * 2
    b_height = 30
    b_unit = None
    b_timestep = None
    
    cap = None
    duration = None
    
    is_playing = False
    
    t_values = []
    t_headings = ['Action', 'Start', 'End']
    
    conf = load_config()
    c_values = conf['classes']
    
    start = None
    end = None
    action = None
    
    label = [None for i in range(3)] # filler values
    
    mleft = [
        [sg.Image(size=(img_w, img_h), background_color='white', key='-IMAGE-')],
        [sg.FileBrowse('Load video', file_types=(("MP4 files", "*.mp4"),), key='-LOAD-', enable_events=True)],
        [sg.Text('Time:'), sg.Text(key='-SECONDS-'), sg.Text('/'), sg.Text(key='-DURATION-'), sg.Text('Frame:'), sg.Text(key='-COUNTER-')],
    ]
    
    sleft = [
        [sg.Table(t_values, t_headings, num_rows=10, col_widths=[6, 6, 25], key='-TABLE-', enable_events=True)],
        [sg.Button('Export', key='-EXPORT-')],
    ]
    
    sright = [
        [sg.Button('Play', key='-PLAY-'), sg.Button('Restart', key='-RESTART-')],
        [sg.Button('S', key='-S-'), sg.Button('E', key='-E-')],
        [sg.Button('<', key='-PREV-'), sg.Button('>', key='-NEXT-')],
        [sg.Text('What action did you see?')],
        [sg.Combo(c_values, key='-COMBO-', enable_events=True)],
        [sg.Button('+', key='-ADD-'), sg.Button('-', key='-DELETE-')],
    ]
    
    mright = [
        [sg.Text('Currently playing:'), sg.Text('', key='-VIDEO-')],
        [sg.Column(sleft), sg.Column(sright)]
    ]
    
    layout = [
        [sg.Column(mleft), sg.Column(mright)],
        [sg.Graph((b_width, b_height), (0,0), (b_width, b_height),  background_color='lime', key='-PROGRESS-')],
    ]

    window = sg.Window('App', layout, finalize=True)

    while True:
        event, values = window.read(timeout=0)
        
        if event == sg.WIN_CLOSED:
            break
        
        if event == '-LOAD-':
            cap = load_video(values['-LOAD-'], conf)
            window['-SECONDS-'].update(f'0 s')
            window['-VIDEO-'].update(Path(values['-LOAD-']).name)
            load_from_beginning(window, cap, img_w, img_h)
            update_duration(window, cap)
            update_counter(window, cap)
            b_unit = int(b_width / cap.get(cv2.CAP_PROP_FRAME_COUNT))
            
        if event == '-PLAY-':
            is_playing = not is_playing
            update_play_button(window, is_playing)
            
        if event == '-RESTART-':
            load_from_beginning(window, cap, img_w, img_h)
            update_seconds(window, cap)
            update_counter(window, cap)
            is_playing = False
            update_play_button(window, is_playing)
            
        if event == '-PREV-':
            if get_pos(cap) == 1:
                sg.popup('Invalid step!')
            else:
                load_prev(window, cap, img_w, img_h)
                update_seconds(window, cap)
                update_counter(window, cap)
                is_playing = False
                update_play_button(window, is_playing)
            
        if event == '-NEXT-':
            if get_pos(cap) == cap.get(cv2.CAP_PROP_FRAME_COUNT):
                sg.popup('Invalid step!')
            else:
                load_next(window, cap, img_w, img_h)
                update_seconds(window, cap)
                update_counter(window, cap)
                is_playing = False
                update_play_button(window, is_playing)
            
        if event == '-S-':
            label[0] = get_pos(cap)
            draw_timestep_on_progressbar(window, cap, b_unit, b_height, 'S', conf['start color'])
            
        if event == '-E-':
            label[1] = get_pos(cap)
            draw_timestep_on_progressbar(window, cap, b_unit, b_height, 'E', conf['end color'])
            
        if event == '-COMBO-':
            label[2] = values['-COMBO-']
            
        if event == '-ADD-':
            if label[0] and label[1] and label[0] > 0 and label[1] > 0:
                if type(label[2]) is str:
                    t_values.append(label)
                    update_table(window, t_values)
                else:
                    sg.popup('No label selected!')
            else:
                sg.popup('Wrong label format!')
        
        if is_playing:
            update_seconds(window, cap)
            update_image(window, cap, img_w, img_h)
            update_counter(window, cap)
            
        if cap and get_pos(cap) == cap.get(cv2.CAP_PROP_FRAME_COUNT):
            is_playing = False
            sg.popup('No more frames left!')
            
    window.close()
    
app()